In [1]:
import numpy as np 
import pandas as p
import matplotlib.pyplot as plt 
%matplotlib inline 



In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier


In [3]:
iris = load_iris()
x = iris["data"][:,2:]
y = iris.target

In [4]:
tree_clf = DecisionTreeClassifier(max_depth =2)
tree_clf.fit(x,y)

DecisionTreeClassifier(max_depth=2)

In [5]:
tree_clf.predict_proba([[5,1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [6]:
tree_clf.predict([[5,1.5]])

array([1])

In [7]:
#regression 

In [8]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth = 2)
tree_reg.fit(x,y)

DecisionTreeRegressor(max_depth=2)

## exercise

In [9]:
from sklearn.datasets import make_moons
moons = make_moons(n_samples = 10000,noise = 0.4)


In [41]:
from sklearn.model_selection import train_test_split
x = moons[0]
y = moons[1]
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size =0.2)


In [42]:
dc = DecisionTreeClassifier(random_state = 9)


In [43]:
#verbose parameter controls how much infoto give in output ,more means increases
from sklearn.model_selection import GridSearchCV
param_distributions = {"max_leaf_nodes":list(range(1,100)),"min_samples_split":[2,3,4]}
gs= GridSearchCV(dc,param_distributions,cv = 3,verbose =1)


In [44]:
gs.fit(x_train,y_train)

Fitting 3 folds for each of 297 candidates, totalling 891 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
c:\users\govind\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\govind\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\govind\appdata\local\programs\python\python38\lib\site-packages\sklearn\tree\_classes.py", line 890, in fit
    super().fit(
  File "c:\users\govind\appdata\local\programs\python\python38\lib\site-packages\sklearn\tree\_classes.py", line 284, in fit
    raise ValueError(("max_leaf_nodes {0} must be either None "
ValueError: max_leaf_nodes 1 must be either None or larger than 1

  warnings.warn("Estimator fit failed. 

[Parallel(n_jobs=1)]: Done 891 out of 891 | elapsed:    7.3s finished


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=9),
             param_grid={'max_leaf_nodes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                            12, 13, 14, 15, 16, 17, 18, 19, 20,
                                            21, 22, 23, 24, 25, 26, 27, 28, 29,
                                            30, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [45]:
gs.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=26, random_state=9)

In [46]:
from sklearn.metrics import accuracy_score

y_pred = gs.predict(x_test)
accuracy_score(y_test, y_pred)

0.853

In [47]:
gs.best_estimator_.fit(x_train,y_train)
y_pred = gs.best_estimator_.predict(x_train)
accuracy_score(y_train,y_pred)

0.8645

In [48]:
y_pred = gs.best_estimator_.predict(x_test)
accuracy_score(y_test,y_pred)

0.853

In [18]:
#8

In [49]:
'''8.
Exercise: Grow a forest.

a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn's ShuffleSplit class for this.'''

"8.\nExercise: Grow a forest.\n\na. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn's ShuffleSplit class for this."

In [50]:
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits = 1000,random_state =3,test_size= len(x_train)-100)

In [51]:
#addition of test  and train ratio would be equal to lenth of data...it justs redhuffles and split
#does not reduce the size of data

In [52]:
s = ss.split(x_train)
t = []
for train_index,test_index in s:
    x_tn = x_train[train_index]
    y_tn = y_train[train_index]
    t.append((x_tn,y_tn))
    

In [53]:
from sklearn.base import clone

forest = [clone(gs.best_estimator_) for _ in range(1000)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, t):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(x_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.785989

In [54]:
'''accuracy_scores=[]
for (i,j) in t:
        gs.fit(i,j)
        y_pred =  gs.predict(x_test)
        accuracy_scores.append(accuracy_score(y_test,y_pred))
np.mean(accuracy_scores)'''
    

'accuracy_scores=[]\nfor (i,j) in t:\n        gs.fit(i,j)\n        y_pred =  gs.predict(x_test)\n        accuracy_scores.append(accuracy_score(y_test,y_pred))\nnp.mean(accuracy_scores)'

In [55]:
y_pred = np.empty([1000, len(x_test)], dtype=np.uint8)
for index,pred in enumerate(forest):
    y_pred[index] = pred.predict(x_test)

In [57]:
from scipy.stats import mode
y_modal_value,n_values = mode(y_pred,axis =0)

In [58]:
(y_modal_value,n_values)

(array([[1, 0, 0, ..., 1, 0, 0]], dtype=uint8),
 array([[816, 988, 960, ..., 840, 921, 955]]))

In [59]:
accuracy_score(y_test,y_modal_value.reshape([-1,1]))

0.853